In [1]:
import numpy as np
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import math
from data_partition import DistributedDataSet
from init_ipfs_nodes import create_ipfs_nodes
import ipfshttpclient
import piskg
import copy
import multiprocessing
from multiprocessing.pool import ThreadPool
from datetime import datetime
from datetime import timedelta
import json
import time
from web3 import Web3, HTTPProvider
import json

In [2]:
def generate_random_data(data_size, n_classes, seed):
    random.seed(seed)
    x_data = pd.DataFrame([[random.random() for _ in range(data_size)], [random.random() for _ in range(data_size)]]).T
    y_data = pd.DataFrame([(list(np.arange(n_classes))*int(data_size/n_classes))])
    data = pd.DataFrame(np.concatenate((x_data.values, y_data.values.T), axis=1))
    data.columns = ['x1', 'x2', 'y']
    return data

In [3]:
# CLIENTS = 8 #don't set more than 9 because of IPFS initialization script
# CLIENT_GRAPH = {
#     0: [1, 4],
#     1: [0, 5, 2],
#     2: [3, 1],
#     3: [2, 4, 6],
#     4: [3, 7, 0],
#     5: [1],
#     6: [3],
#     7: [4]
# }

CLIENTS = 4 #don't set more than 9 because of IPFS initialization script
CLIENT_GRAPH = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: [1, 2],
}

SEED = 0
ROUNDS = 5
BATCH_SIZE = 45

#supports only pandas dataframe for now. 
#input values should be labelled as x0, x1, etc. 
#output values should be labelled as y
DATASET = generate_random_data(2000, 5, SEED)

server = Web3(HTTPProvider('http://localhost:8545'))
CONTRACT_ADDRESS = server.toChecksumAddress("0xdcf23f956e84b892efbf75f029478c6f75f196e5")
DEFAULT_ADDRESS  = server.toChecksumAddress(server.eth.accounts[0])
with open('../blkTrial/HelloWorld/build/contracts/Model.json') as f:
    CONTRACT_DATA = json.load(f)
CONTRACT_ABI = CONTRACT_DATA['abi']
BLKCHAIN_ACCOUNTS = server.eth.accounts

In [4]:
dd = DistributedDataSet(DATASET, SEED, BATCH_SIZE, CLIENTS)
df_di, test = dd.get_distributed_dataset(0.1)

In [5]:
create_ipfs_nodes(CLIENTS)

generating ED25519 keypair...done
peer identity: 12D3KooWCxoxM2A4VBEKRmbdo3pDubuRZgRq5okWDr5CRNDMzAAa
initializing IPFS node at /home/hitesh/.ipfs_fl0
to get started, enter:

	ipfs cat /ipfs/QmQPeNsJPyVWPFDVHb77w8G42Fvo15z4bG2X8D2GhfbSXc/readme

removed /dnsaddr/bootstrap.libp2p.io/p2p/QmNnooDu7bfjPFoTZYxMNLWUQJyrVwtbZg5gBMjTezGAJN
removed /dnsaddr/bootstrap.libp2p.io/p2p/QmQCU2EcMqAqQPR2i9bChDtGNJchTbq5TbXJJ16u19uLTa
removed /dnsaddr/bootstrap.libp2p.io/p2p/QmbLHAnMoJPWSCR5Zhtx6BHJX9KiKNN6tpvbUcqanj75Nb
removed /dnsaddr/bootstrap.libp2p.io/p2p/QmcZf59bWwK5XFi76CZX8cbJ4BhTzzA3gU1ZjYZcYW3dwt
removed /ip4/104.131.131.82/tcp/4001/p2p/QmaCpDMGvV2BGHeYERUEnRQAwe3N8SzbUtfsmvsqQLuvuJ
removed /ip4/104.131.131.82/udp/4001/quic/p2p/QmaCpDMGvV2BGHeYERUEnRQAwe3N8SzbUtfsmvsqQLuvuJ
generating ED25519 keypair...done
peer identity: 12D3KooWEcYTmjgb5avwkDv3uggmtRbUmAnQDyem7x5GDDLsi8JV
initializing IPFS node at /home/hitesh/.ipfs_fl1
to get started, enter:

	ipfs cat /ipfs/QmQPeNsJPyVWPFDVHb77w8G42Fvo15

In [6]:
# class ClientNetwork:
#     def __init__(self, count, structure, ipfs, blkchain, train_data, test_data):
#         self.count = count
#         self.structure = structure
#         self.clients = ipfs
#         self.blkchain = blkchain
#         self.train_data = train_data
#         self.test_data = test_data
        
#         self.clients = []
        
#     def start_clients(self):
#         for i in range(self.count):
#             ag = Client(i, self.structure[i], sekf.train_data[i], self.ipfs, self.blkchain, self.test)
#             self.clients.append(ag)
            

In [7]:
def default(obj):
    if type(obj).__module__ == np.__name__:
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj.item()
    else:
        return obj.total_seconds()
#     raise TypeError('Unknown type:', type(obj))

In [8]:
def getTime(messages):
    simulated_communication_times = {i: messages[i]['time'] for i in range(len(messages))}
    slowest_client = max(simulated_communication_times, key=simulated_communication_times.get)
    simulated_time = simulated_communication_times[slowest_client]  # simulated time it would take for server to receive all values
    return simulated_time

In [9]:
class Message:
    def __init__(self, sender, receiver, body):
        self.sender = sender
        self.receiver = receiver
        self.body = body

In [10]:
class Client:
    def __init__(self, client_id, neighbour_ids, client_datasets, client_ipfs_node, client_blkchain_node, test_dataset):
        
        self.client_id = client_id
        self.neighbour_ids = neighbour_ids
        self.neighbour_ids.append(self.client_id) #neighbour includes itself
        self.client_datasets = client_datasets
        self.client_ipfs_node = client_ipfs_node
        self.client_blkchain_node = client_blkchain_node
        self.test_dataset = test_dataset
        
        self.personal_weights = {}
        self.personal_intercepts = {}
        
        self.federated_weights = {}
        self.federated_intercepts = {}
        
        self.personal_accuracy = {}
        self.federated_accuracy = {}
        
        self.train_time = {}
        
        self.nb_weights = {}
        self.nb_intercepts = {}
        
        self.ipfsHash = {}
        
        self.nb_hash = {}
        
        self.blkchain_update = {}
    
    def addModelIpfs(self, round_it):
        start_time = datetime.now()
        model = {
                 'weights': self.personal_weights[round_it][len(self.client_datasets) - 1],
                 'intercepts': self.personal_intercepts[round_it][len(self.client_datasets) - 1],
                 'time': self.train_time[round_it]
                }
        with open('model'+str(self.client_id)+'.json', 'w') as f:
            json.dump(model, f, default=default)

        cmd = "IPFS_PATH=~/"+self.client_ipfs_node+" ipfs add model"+str(self.client_id)+".json"
        
        try:
            modelHash = (os.popen(cmd).read()).strip()
            modelHash = modelHash.split()[1]
            self.ipfsHash[round_it] = modelHash
            stop_time = datetime.now()
            ipfs_time = (stop_time - start_time)
            return modelHash
        except:
            stop_time = datetime.now()
            ipfs_time = (stop_time - start_time)
            return 0
        
    def recModelIpfs(self, round_it, ipfs_hash):
#         try:
            cmd = "IPFS_PATH=~/"+self.client_ipfs_node+" ipfs cat "+ipfs_hash
            model = (os.popen(cmd).read()).strip()
            start_time = datetime.now()
            model = json.loads(os.popen(cmd).read())
            stop_time = datetime.now()
            ipfs_time = (stop_time - start_time)
            return model
#         except KeyError:
#             print(self.client_ipfs_node, round_it)
#             return None
        
    def modelChecker(self, round_it):
        try:
            a = self.ipfsHash[round_it]
            return None
        except KeyError:
            return self.client_id
    
    def client_modelCheck_caller(self, deets):
        client_instance, msg = deets
        round_it = msg.body['round']
        m = client_instance.modelChecker(round_it)
        return m
    
#     def client_model_caller(self, deets):
#         client_instance, msg = deets
#         round_it = msg.body['round']
#         m = client_instance.recModelIpfs(round_it)
#         return m

    def get_ipfs_hash(self, target_node_blk, round_it, target_node_id):
        contract = server.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)
        msg = contract.functions.getIpfsHashForUser(target_node_blk, str(round_it)).call()
        if(msg==''):
            print("Round "+str(round_it)+" missing on blockchain for Node "+str(target_node_id))
            return 0
        else:
            return (msg)
            

    def client_model_caller(self, deets):
        client_instance, msg = deets
        round_it = msg.body['round']
        target_node_blk = client_instance.client_blkchain_node
        target_node_id = client_instance.client_id
        ipfs_hash = self.get_ipfs_hash(target_node_blk, round_it, target_node_id)
        if(ipfs_hash!=0):
            m = self.recModelIpfs(round_it, ipfs_hash)
            return m
        else:
            print("Couldn't retreive ipfs hash from blockchain.")
    
    def client_train_caller(self, deets):
        client_instance, msg = deets
#         print("Training Caller: ", client_instance.client_id)
        if(client_instance.client_id == self.client_id):
            self.train_client(msg)
        else:  
            client_instance.train_client(msg)
#         print("Training Caller 2: ", client_instance.client_id)
            
    
    def get_nb_weights(self, round_it):
        m = multiprocessing.Manager()
        lock = m.Lock()
        
        with ThreadPool(len(self.neighbour_ids)) as calling_pool:
            args = []
            for cl in (self.neighbour_ids):
                body = {'round': round_it, 'lock': lock}
                msg = Message(self.client_id, cl, body)
                args.append((allCl.clients[cl], msg))
            msgs = calling_pool.map(self.client_modelCheck_caller, args)
            msgs = np.array(msgs)
        
        untrained_nb = list(msgs[msgs!=None])
        print("Current Node: ", self.client_id)
        print("Untrained Neighbour: ", untrained_nb)
        print("All neighbours: ", self.neighbour_ids)
        if(len(untrained_nb)!=0):
            with ThreadPool(len(untrained_nb)) as calling_pool:
                args = []
                for cl in (untrained_nb):
                    body = {'round': round_it, 'lock': lock}
                    msg = Message(self.client_id, cl, body)
                    args.append((allCl.clients[cl], msg))
                calling_pool.map(self.client_train_caller, args)
                
        with ThreadPool(len(self.neighbour_ids)) as calling_pool:
            args = []
            for cl in (self.neighbour_ids):
                body = {'round': round_it, 'lock': lock}
                msg = Message(self.client_id, cl, body)
                args.append((allCl.clients[cl], msg))
            msgs = calling_pool.map(self.client_model_caller, args)        
        
        time_start = datetime.now()
        train_time = getTime(msgs)
        
        self.nb_weights[round_it] = []
        self.nb_intercepts[round_it] = []
        
        for msg in msgs:
            self.nb_weights[round_it].append(msg['weights'])
            self.nb_intercepts[round_it].append(msg['intercepts'])
        
        time_stop = datetime.now()
        fedtime = time_stop - time_start
        totTime = fedtime+timedelta(train_time)
        self.FedAvg(round_it)
#         return msgs 
    
    def FedAvg(self, round_it):
        self.federated_weights[round_it] = np.average(self.nb_weights[round_it], axis=0)
        self.federated_intercepts[round_it] = np.average(self.nb_intercepts[round_it], axis=0)
    
    def train_client(self, message):
            training_flag = 0
            training_over = 0
            print("Training node ", self.client_id)
            round_it = message.body['round']
            start_time = datetime.now()
            for epoch in range(len(self.client_datasets)):
                if(training_flag==0):
                    X = self.client_datasets[epoch].drop('y', axis=1)
                    y = self.client_datasets[epoch]['y']
                    weights, intercepts = self.compute_epoch(X, y, round_it, epoch, message.sender, self.client_id)

                    if(epoch==0):
                        try:
                            self.personal_weights[round_it]
                            training_flag = 1
                        except:
                            pass
                            self.personal_weights[round_it] = []
                            self.personal_intercepts[round_it] = []

                    if(training_flag==0):
                        self.personal_weights[round_it].append(weights)
                        self.personal_intercepts[round_it].append(intercepts)

            if(training_flag==0):
                stop_time = datetime.now()
                comp_time = stop_time - start_time
                self.train_time[round_it] = comp_time
                success = self.addModelIpfs(round_it)
#                 print("Training finished for ", self.client_id)
                if (success!=0):
                    contract = server.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)
                    tx_hash = contract.functions.addFile(str(round_it), success).transact({'from':self.client_blkchain_node})
                    rec = server.eth.waitForTransactionReceipt(tx_hash)
                    if(rec['status']!=1):
                        print('TRANSACTION REVERTED BY BLOCKCHAIN')
                    else:
                        self.blkchain_update[round_it] = 1
            else:
                while(training_over==0):
                    try:
                        self.blkchain_update[round_it]
                        training_over = 1
#                         print("Training over")
                    except:
                        time.sleep(1)
                        
#                 print("Client ", self.client_id, " training already.")
            
            
                    
    
    def compute_epoch(self, X, y, round_it, epoch, sender, receiver):            
            
        lr = SGDClassifier(alpha=0.0001, loss="log", random_state=0)

        if (epoch >= 1):
            try:
                weights = copy.deepcopy(self.personal_weights[round_it][epoch - 1])
                intercepts = copy.deepcopy(self.personal_intercepts[round_it][epoch - 1])
            except:
                print("Node: ", self.client_id)
                print("Epoch: ", epoch-1)
                print("Round: ", round_it)
                time.sleep(3)
                weights = copy.deepcopy(self.personal_weights[round_it][epoch - 1])
                intercepts = copy.deepcopy(self.personal_intercepts[round_it][epoch - 1])
                
        else:
            if(round_it>=1):
                try:
                    weights = copy.deepcopy(self.federated_weights[round_it - 1])
                    intercepts = copy.deepcopy(self.federated_intercepts[round_it - 1])
                    print("Federated weights set for epoch ", epoch, " in round  ", round_it)
                except: 
                    rounds_trained = len(self.nb_weights)
                    while(rounds_trained<round_it):
                        self.get_nb_weights(rounds_trained)
                        rounds_trained+=1
                    
                    try:
                        weights = copy.deepcopy(self.federated_weights[round_it - 1])
                        intercepts = copy.deepcopy(self.federated_intercepts[round_it - 1])
                    except:
#                         print("Round: ", round_it)
#                         print("Node: ", self.client_id)
#                         print("Maxi var: ", maxi)
                        weights = copy.deepcopy(self.federated_weights[round_it - 1])
                        intercepts = copy.deepcopy(self.federated_intercepts[round_it - 1])
            else:
                weights = None
                intercepts = None
                
        lr.fit(X, y, coef_init=weights, intercept_init=intercepts)
        local_weights = lr.coef_
        local_intercepts = lr.intercept_
        
        if(epoch==(len(self.client_datasets) - 1)):
            acc = lr.score(self.test_dataset[['x1', 'x2']], self.test_dataset['y'])
            self.personal_accuracy[round_it] = acc
            
        return local_weights, local_intercepts
        
#     def federated_averaging(self):
        

In [11]:
class AllClients:
    def __init__(self, clients):
        self.clients = clients
        self.training = {int(client): False for client in range(CLIENTS)}
        with open('training.json', 'w') as f:
            json.dump(self.training, f)

In [12]:
client_li = []
for i in range(CLIENTS):
    ag = Client(i, CLIENT_GRAPH[i], df_di[i], '.ipfs_fl'+str(i), BLKCHAIN_ACCOUNTS[i], test)
    client_li.append(ag)

allCl = AllClients(client_li)

In [13]:
allCl.clients[0].get_nb_weights(0)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0


 354 B / 354 B  100.00%

In [14]:
def printLengths(clients, round_it):
    for i in range(len(clients.clients)):
        try:
            print(i,' ',len(clients.clients[i].personal_intercepts[round_it]))
        except:
            print(i,'  0')

In [15]:
printLengths(allCl, 0)

0   10
1   10
2   10
3   0


In [16]:
allCl.clients[0].get_nb_weights(1)
printLengths(allCl, 0)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  Training node  0
Federated weights set for epoch  20  in round   1

Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Current Node:  2Training node  3

Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3


 359 B / 359 B  100.00%

0   10
1   10
2   10
3   10


In [17]:
allCl.clients[0].get_nb_weights(2)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0
Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Current Node:  2Training node  3

Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3
Current Node:  3Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]

Untrained Neighbour:  []
All neighbours:  [1, 2, 3]


 354 B / 354 B  100.00%

In [18]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   0

0   0
1   0
2   0
3   0


In [19]:
allCl.clients[0].get_nb_weights(3)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0
Current Node:  2
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3
Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Training node  3
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]


 354 B / 354 B  100.00%

In [22]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   0


In [23]:
allCl.clients[0].get_nb_weights(4)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0
Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Current Node:  2
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3
Training node  3
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]
Training finished for  0


 358 B / 358 B  100.00%

Training Caller 2:  0


 355 B / 355 B  100.00%

Training finished for  3
Training Caller 2:  3
Training over
Client  3  training already.
Training Caller 2:  3


 354 B / 354 B  100.00%

Training finished for  2
Training finished for  1
Training Caller 2:  2
Training Caller 2:  1


In [24]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)
print()
printLengths(allCl, 4)

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10



In [25]:
allCl.clients[0].get_nb_weights(5)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0
Current Node:  1
Current Node:  2
Untrained Neighbour: Untrained Neighbour:  [3]
All neighbours:   [3]
All neighbours:  [0, 3, 2]
[0, 3, 1]
Training node  3
Training node  3
Current Node: Current Node:   3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]3
Untrained Neighbour:  []
All neighbours:  
[1, 2, 3]
Training finished for  0


 358 B / 358 B  100.00%

Training Caller 2:  0
Training finished for  3


 355 B / 355 B  100.00%

Training Caller 2:  3


 354 B / 354 B  100.00%

Training finished for  1
Training Caller 2:  1
Training over
Client  3  training already.
Training Caller 2:  3


 360 B / 360 B  100.00%

Training finished for  2
Training Caller 2:  2


In [27]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)
print()
printLengths(allCl, 4)
print()
printLengths(allCl, 5)

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   0


In [27]:
allCl.clients[0].federated_intercepts

{0: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159]),
 1: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159]),
 2: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159]),
 3: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159])}

In [24]:
allCl.clients[3].personal_accuracy

{0: 0.19}

In [17]:
allCl.clients[0].nb_intercepts[0]

[[-9.08828927688947,
  -1.897113601450453,
  3.324919978719681,
  -3.607171033290969,
  -5.6914308345434925],
 [-2.7825014532674115,
  -1.9163357277199058,
  1.4142788785330778,
  7.635749088239045,
  -9.203838156493957],
 [-1.4408951288098966,
  -3.6041663195640137,
  3.4038328221333707,
  -6.522774884794829,
  0.35369420967760234]]

In [20]:
allCl.clients[0].nb_intercepts[1]

[[-9.08828927688947,
  -1.897113601450453,
  3.324919978719681,
  -3.607171033290969,
  -5.6914308345434925],
 [-2.7825014532674115,
  -1.9163357277199058,
  1.4142788785330778,
  7.635749088239045,
  -9.203838156493957],
 [-1.4408951288098966,
  -3.6041663195640137,
  3.4038328221333707,
  -6.522774884794829,
  0.35369420967760234]]

In [10]:
def client_computation_caller(inp):
    client_instance, body = inp
    return_message = client_instance.train_client(body['round'])
    return return_message

In [11]:
for ro in range(ROUNDS):
    m = multiprocessing.Manager()
    lock = m.Lock()
    with ThreadPool(CLIENTS) as calling_pool:
        args = []
        for cl in range(CLIENTS):
            body = {'round': ro, 'lock': lock}
            args.append((allCl.clients[cl], body))
        calling_pool.map(client_computation_caller, args)

In [46]:
allCl.clients[1].personal_intercepts[0]

[array([-1.19024283, -8.22172283, -9.42661473, -3.97311779, -4.33640538]),
 array([-10.33395532,   3.24845949,   5.356186  ,  -9.3852317 ,
         -2.30635674]),
 array([-10.55145539, -12.46190679,   0.64291909,  -1.13745339,
         -0.18818607]),
 array([  0.52149334, -16.64293375,  -6.27774062,  -0.99584345,
         -0.70731457]),
 array([-1.44089513, -3.60416632,  3.40383282, -6.52277488,  0.35369421])]